# University Database

This tutorial builds a complete university registration system to demonstrate:

- **Schema design** with realistic relationships
- **Data population** using Faker for synthetic data
- **Rich query patterns** from simple to complex

University databases are classic examples because everyone understands students, courses, enrollments, and grades. The domain naturally demonstrates:

- One-to-many relationships (department → courses)
- Many-to-many relationships (students ↔ courses via enrollments)
- Workflow dependencies (enrollment requires both student and section to exist)

In [ ]:
import datajoint as dj
import numpy as np
from datetime import date

schema = dj.Schema('tutorial_university')

## Schema Design

Our university schema models:

| Table | Purpose |
|-------|--------|
| `Student` | Student records with contact info |
| `Department` | Academic departments |
| `StudentMajor` | Student-declared majors |
| `Course` | Course catalog |
| `Term` | Academic terms (Spring/Summer/Fall) |
| `Section` | Course offerings in specific terms |
| `Enroll` | Student enrollments in sections |
| `LetterGrade` | Grade scale (lookup) |
| `Grade` | Assigned grades |

In [ ]:
@schema
class Student(dj.Manual):
    definition = """
    student_id : uint32           # university-wide ID
    ---
    first_name : varchar(40)
    last_name : varchar(40)
    sex : enum('F', 'M', 'U')
    date_of_birth : date
    home_city : varchar(60)
    home_state : char(2)          # US state code
    """

In [ ]:
@schema
class Department(dj.Manual):
    definition = """
    dept : varchar(6)   # e.g. BIOL, CS, MATH
    ---
    dept_name : varchar(200)
    """

In [ ]:
@schema
class StudentMajor(dj.Manual):
    definition = """
    -> Student
    ---
    -> Department
    declare_date : date
    """

In [ ]:
@schema
class Course(dj.Manual):
    definition = """
    -> Department
    course : uint32               # course number, e.g. 1010
    ---
    course_name : varchar(200)
    credits : decimal(3,1)
    """

In [ ]:
@schema
class Term(dj.Manual):
    definition = """
    term_year : year
    term : enum('Spring', 'Summer', 'Fall')
    """

In [ ]:
@schema
class Section(dj.Manual):
    definition = """
    -> Course
    -> Term
    section : char(1)
    ---
    auditorium : varchar(12)
    """

In [ ]:
@schema
class Enroll(dj.Manual):
    definition = """
    -> Student
    -> Section
    """

In [ ]:
@schema
class LetterGrade(dj.Lookup):
    definition = """
    grade : char(2)
    ---
    points : decimal(3,2)
    """
    contents = [
        ['A',  4.00], ['A-', 3.67],
        ['B+', 3.33], ['B',  3.00], ['B-', 2.67],
        ['C+', 2.33], ['C',  2.00], ['C-', 1.67],
        ['D+', 1.33], ['D',  1.00],
        ['F',  0.00]
    ]

In [ ]:
@schema
class Grade(dj.Manual):
    definition = """
    -> Enroll
    ---
    -> LetterGrade
    """

In [ ]:
dj.Diagram(schema)

## Populate with Synthetic Data

We use [Faker](https://faker.readthedocs.io/) to generate realistic student data.

In [ ]:
import faker
import random

fake = faker.Faker()
faker.Faker.seed(42)
random.seed(42)

In [ ]:
def generate_students(n=500):
    """Generate n student records."""
    fake_name = {'F': fake.name_female, 'M': fake.name_male}
    for student_id in range(1000, 1000 + n):
        sex = random.choice(['F', 'M'])
        name = fake_name[sex]().split()[:2]
        yield {
            'student_id': student_id,
            'first_name': name[0],
            'last_name': name[-1],
            'sex': sex,
            'date_of_birth': fake.date_between(start_date='-35y', end_date='-17y'),
            'home_city': fake.city(),
            'home_state': fake.state_abbr()
        }

Student.insert(generate_students(500))
print(f"Inserted {len(Student())} students")

In [ ]:
# Departments
Department.insert([
    {'dept': 'CS', 'dept_name': 'Computer Science'},
    {'dept': 'BIOL', 'dept_name': 'Life Sciences'},
    {'dept': 'PHYS', 'dept_name': 'Physics'},
    {'dept': 'MATH', 'dept_name': 'Mathematics'},
])

# Assign majors to ~75% of students
students = Student.keys()
depts = Department.keys()
StudentMajor.insert(
    {**s, **random.choice(depts), 'declare_date': fake.date_between(start_date='-4y')}
    for s in students if random.random() < 0.75
)
print(f"{len(StudentMajor())} students declared majors")

In [ ]:
# Course catalog
Course.insert([
    ['BIOL', 1010, 'Biology in the 21st Century', 3],
    ['BIOL', 2020, 'Principles of Cell Biology', 3],
    ['BIOL', 2325, 'Human Anatomy', 4],
    ['BIOL', 2420, 'Human Physiology', 4],
    ['PHYS', 2210, 'Physics for Scientists I', 4],
    ['PHYS', 2220, 'Physics for Scientists II', 4],
    ['PHYS', 2060, 'Quantum Mechanics', 3],
    ['MATH', 1210, 'Calculus I', 4],
    ['MATH', 1220, 'Calculus II', 4],
    ['MATH', 2270, 'Linear Algebra', 4],
    ['MATH', 2280, 'Differential Equations', 4],
    ['CS', 1410, 'Intro to Object-Oriented Programming', 4],
    ['CS', 2420, 'Data Structures & Algorithms', 4],
    ['CS', 3500, 'Software Practice', 4],
    ['CS', 3810, 'Computer Organization', 4],
])
print(f"{len(Course())} courses in catalog")

In [ ]:
# Academic terms 2020-2024
Term.insert(
    {'term_year': year, 'term': term}
    for year in range(2020, 2025)
    for term in ['Spring', 'Summer', 'Fall']
)

# Create sections for each course-term with 1-3 sections
for course in Course.keys():
    for term in Term.keys():
        for sec in 'abc'[:random.randint(1, 3)]:
            if random.random() < 0.7:  # Not every course offered every term
                Section.insert1({
                    **course, **term,
                    'section': sec,
                    'auditorium': f"{random.choice('ABCDEF')}{random.randint(100, 400)}"
                }, skip_duplicates=True)

print(f"{len(Section())} sections created")

In [ ]:
# Enroll students in courses
terms = Term.keys()
for student in Student.keys():
    # Each student enrolls over 2-6 random terms
    student_terms = random.sample(terms, k=random.randint(2, 6))
    for term in student_terms:
        # Take 2-4 courses per term
        available = (Section & term).keys()
        if available:
            for section in random.sample(available, k=min(random.randint(2, 4), len(available))):
                Enroll.insert1({**student, **section}, skip_duplicates=True)

print(f"{len(Enroll())} enrollments")

In [ ]:
# Assign grades to ~90% of enrollments (some incomplete)
grades = LetterGrade.to_arrays('grade')
# Weight toward B/C range
weights = [5, 8, 10, 15, 12, 10, 15, 10, 5, 5, 5]

for enroll in Enroll.keys():
    if random.random() < 0.9:
        Grade.insert1({**enroll, 'grade': random.choices(grades, weights=weights)[0]})

print(f"{len(Grade())} grades assigned")

## Querying Data

DataJoint queries are composable expressions. Displaying a query shows a preview; use `fetch()` to retrieve data.

In [ ]:
dj.config['display.limit'] = 8  # Limit preview rows

### Restriction (`&` and `-`)

Filter rows using `&` (keep matching) or `-` (remove matching).

In [ ]:
# Students from California
Student & {'home_state': 'CA'}

In [ ]:
# Female students NOT from California
(Student & {'sex': 'F'}) - {'home_state': 'CA'}

In [ ]:
# SQL-style string conditions
Student & 'home_state IN ("CA", "TX", "NY")'

In [ ]:
# OR conditions using a list
Student & [{'home_state': 'CA'}, {'home_state': 'TX'}]

### Subqueries in Restrictions

Use another query as a restriction condition.

In [ ]:
# Students majoring in Computer Science
Student & (StudentMajor & {'dept': 'CS'})

In [ ]:
# Students who have NOT taken any Math courses
Student - (Enroll & {'dept': 'MATH'})

In [ ]:
# Students with ungraded enrollments (enrolled but no grade yet)
Student & (Enroll - Grade)

In [ ]:
# All-A students: have grades AND no non-A grades
all_a = (Student & Grade) - (Grade - {'grade': 'A'})
all_a

### Projection (`.proj()`)

Select, rename, or compute attributes.

In [ ]:
# Select specific attributes
Student.proj('first_name', 'last_name')

In [ ]:
# Computed attribute: full name
Student.proj(full_name="CONCAT(first_name, ' ', last_name)")

In [ ]:
# Calculate age in years
Student.proj('first_name', 'last_name', 
             age='TIMESTAMPDIFF(YEAR, date_of_birth, CURDATE())')

In [ ]:
# Keep all attributes plus computed ones with ...
Student.proj(..., age='TIMESTAMPDIFF(YEAR, date_of_birth, CURDATE())')

In [ ]:
# Exclude specific attributes with -
Student.proj(..., '-date_of_birth')

In [ ]:
# Rename attribute
Student.proj('first_name', family_name='last_name')

### Universal Set (`dj.U()`)

The universal set `dj.U()` extracts unique values of specified attributes.

In [ ]:
# All unique first names
dj.U('first_name') & Student

In [ ]:
# All unique home states of enrolled students
dj.U('home_state') & (Student & Enroll)

In [ ]:
# Birth years of students in CS courses
dj.U('birth_year') & (
    Student.proj(birth_year='YEAR(date_of_birth)') & (Enroll & {'dept': 'CS'})
)

### Join (`*`)

Combine tables on matching attributes.

In [ ]:
# Students with their declared majors
Student.proj('first_name', 'last_name') * StudentMajor

In [ ]:
# Courses with department names
Course * Department.proj('dept_name')

In [ ]:
# Left join: all students, including those without majors (NULL for unmatched)
Student.proj('first_name', 'last_name').join(StudentMajor, left=True)

In [ ]:
# Multi-table join: grades with student names and course info
(Student.proj('first_name', 'last_name') 
 * Grade 
 * Course.proj('course_name', 'credits'))

### Aggregation (`.aggr()`)

Group rows and compute aggregate statistics.

In [ ]:
# Number of students per department
Department.aggr(StudentMajor, n_students='COUNT(*)')

In [ ]:
# Breakdown by sex per department
Department.aggr(
    StudentMajor * Student,
    n_female='SUM(sex="F")',
    n_male='SUM(sex="M")'
)

In [ ]:
# Enrollment counts per course (with course name)
Course.aggr(Enroll, ..., n_enrolled='COUNT(*)')

In [ ]:
# Average grade points per course
Course.aggr(
    Grade * LetterGrade,
    'course_name',
    avg_gpa='AVG(points)',
    n_grades='COUNT(*)'
)

### Complex Queries

Combine operators to answer complex questions.

In [ ]:
# Student GPA: weighted average of grade points by credits
student_gpa = Student.aggr(
    Grade * LetterGrade * Course,
    'first_name', 'last_name',
    total_credits='SUM(credits)',
    gpa='SUM(points * credits) / SUM(credits)'
)
student_gpa

In [ ]:
# Top 5 students by GPA (with at least 12 credits)
student_gpa & 'total_credits >= 12' & dj.Top(5, order_by='gpa DESC')

In [ ]:
# Students who have taken courses in ALL departments
# (i.e., no department exists where they haven't enrolled)
all_depts = Student - (
    Student.proj() * Department - Enroll.proj('student_id', 'dept')
)
all_depts.proj('first_name', 'last_name')

In [ ]:
# Most popular courses (by enrollment) per department
course_enrollment = Course.aggr(Enroll, ..., n='COUNT(*)')

# For each department, find the max enrollment
max_per_dept = Department.aggr(course_enrollment, max_n='MAX(n)')

# Join to find courses matching the max
course_enrollment * max_per_dept & 'n = max_n'

In [ ]:
# Grade distribution: count of each grade across all courses
LetterGrade.aggr(Grade, ..., count='COUNT(*)') & 'count > 0'

### Fetching Results

Use the fetch methods to retrieve data into Python:
- `to_dicts()` — list of dictionaries
- `to_arrays()` — numpy arrays
- `to_pandas()` — pandas DataFrame
- `fetch1()` — single row (query must return exactly one row)

In [ ]:
# Fetch as numpy recarray
data = (Student & {'home_state': 'CA'}).to_arrays()
print(f"Type: {type(data).__name__}, shape: {data.shape}")
data[:3]

In [ ]:
# Fetch as list of dicts
(Student & {'home_state': 'CA'}).to_dicts(limit=3)

In [ ]:
# Fetch specific attributes as arrays
first_names, last_names = (Student & {'home_state': 'CA'}).to_arrays('first_name', 'last_name')
list(zip(first_names, last_names))[:5]

In [ ]:
# Fetch single row with fetch1
student = (Student & {'student_id': 1000}).fetch1()
print(f"{student['first_name']} {student['last_name']} from {student['home_city']}, {student['home_state']}")

In [ ]:
# Fetch as pandas DataFrame
(student_gpa & 'total_credits >= 12').to_pandas().sort_values('gpa', ascending=False).head(10)

## Summary

This tutorial demonstrated:

| Operation | Syntax | Purpose |
|-----------|--------|--------|
| Restriction | `A & cond` | Keep matching rows |
| Anti-restriction | `A - cond` | Remove matching rows |
| Projection | `A.proj(...)` | Select/compute attributes |
| Join | `A * B` | Combine tables |
| Left join | `A.join(B, left=True)` | Keep all rows from A |
| Aggregation | `A.aggr(B, ...)` | Group and aggregate |
| Universal | `dj.U('attr') & A` | Unique values |
| Top | `A & dj.Top(n, order_by=...)` | Limit/order results |
| Fetch keys | `A.keys()` | Primary key dicts |
| Fetch arrays | `A.to_arrays(...)` | Numpy arrays |
| Fetch dicts | `A.to_dicts()` | List of dicts |
| Fetch pandas | `A.to_pandas()` | DataFrame |
| Fetch one | `A.fetch1()` | Single row dict |

In [ ]:
# Cleanup
schema.drop(prompt=False)